# 1

In [ ]:
!pip install Pillow tqdm

In [ ]:
import cv2
import os
import numpy as np

from src import model
from src import util
from src.body import Body
from src.hand import Hand


def load_model(use_hand=False):
    model_path = 'model/pose_iter_584000.caffemodel.pt'
    body_estimation = Body(model_path, 'body25')
    if use_hand:
        hand_estimation = Hand('model/hand_pose_model.pth')
    else:
        hand_estimation = None
    return body_estimation, hand_estimation


def inference(oriImg, test_image_path, body_estimation, hand_estimation, output_path='.'):
    candidate, subset = body_estimation(oriImg)
    
    # 검정 배경 생성 (원본 이미지 크기와 동일)
    canvas = np.zeros_like(oriImg)

    # Body pose 그리기
    canvas = util.draw_bodypose(canvas, candidate, subset, 'body25')

    if hand_estimation is not None:
        # Detect hand
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = []
        for x, y, w, is_left in hands_list:
            peaks = hand_estimation(oriImg[y:y+w, x:x+w, :])
            peaks[:, 0] = np.where(
                peaks[:, 0] == 0, peaks[:, 0], peaks[:, 0]+x)
            peaks[:, 1] = np.where(
                peaks[:, 1] == 0, peaks[:, 1], peaks[:, 1]+y)
            all_hand_peaks.append(peaks)
        canvas = util.draw_handpose(canvas, all_hand_peaks)

    # 결과 저장: 저장 이름을 요청한 형식으로 변경
    img_basename = os.path.basename(test_image_path).split('.')[0]  # 파일 이름에서 확장자 제거
    result_path = os.path.join(output_path, f'{img_basename}_rendered.png')
    cv2.imwrite(result_path, canvas)
    print(f"Saved result to: {result_path}")


if __name__ == "__main__":
    body_estimation, hand_estimation = load_model(use_hand=True)
    
    # 입력 및 출력 경로 설정
    input_path = 'input/image'
    output_path = 'pre-processing/openpose_img'
    os.makedirs(output_path, exist_ok=True)

    # 입력 디렉토리 내 모든 이미지 처리
    for test_image_name in os.listdir(input_path):
        if not test_image_name.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
            continue
        
        test_image_path = os.path.join(input_path, test_image_name)
        oriImg = cv2.imread(test_image_path)  # B,G,R order
        
        print(f'Processing: {test_image_path}')
        inference(oriImg, test_image_path, body_estimation, hand_estimation, output_path)


Processing: /Users/parkyunsu/Downloads/00022_00.jpg
Saved result to: pre-processing/openpose_img/00022_00_rendered.png


# Json

In [ ]:
import cv2
import os
import json
import numpy as np
from src import util
from src.body import Body
from src.hand import Hand


def load_model(use_hand=False):
    model_path = 'model/pose_iter_584000.caffemodel.pt'
    body_estimation = Body(model_path, 'body25')
    hand_estimation = Hand('model/hand_pose_model.pth') if use_hand else None
    return body_estimation, hand_estimation


def inference_and_save_json(image_path, body_estimation, hand_estimation, output_path):
    oriImg = cv2.imread(image_path)  # B,G,R order
    if oriImg is None:
        print(f"Error: Could not read image from path {image_path}")
        return

    # Body estimation
    candidate, subset = body_estimation(oriImg)

    # Hand estimation
    all_hand_peaks = []
    if hand_estimation is not None:
        hands_list = util.handDetect(candidate, subset, oriImg)
        for x, y, w, is_left in hands_list:
            peaks = hand_estimation(oriImg[y:y+w, x:x+w, :])
            peaks[:, 0] = np.where(peaks[:, 0] == 0, peaks[:, 0], peaks[:, 0]+x)
            peaks[:, 1] = np.where(peaks[:, 1] == 0, peaks[:, 1], peaks[:, 1]+y)
            all_hand_peaks.append(peaks)

    # Output file paths
    img_basename = os.path.basename(image_path).split('.')[0]  # ex: 00022_00
    json_filename = f"{img_basename}_keypoints.json"
    json_path = os.path.join(output_path, json_filename)

    # Prepare JSON output
    json_data = {"version": 1.3, "people": []}
    for person in subset:
        person_data = {
            "person_id": [-1],
            "pose_keypoints_2d": [],
            "face_keypoints_2d": [],
            "hand_left_keypoints_2d": [],
            "hand_right_keypoints_2d": [],
            "pose_keypoints_3d": [],
            "face_keypoints_3d": [],
            "hand_left_keypoints_3d": [],
            "hand_right_keypoints_3d": []
        }

        # Add body keypoints
        for idx in range(len(candidate)):
            if idx in person[:len(candidate)]:
                keypoint = candidate[int(idx)]
                person_data["pose_keypoints_2d"].extend([float(keypoint[0]), float(keypoint[1]), float(keypoint[2])])
            else:
                person_data["pose_keypoints_2d"].extend([0.0, 0.0, 0.0])  # Default for missing points

        # Add hand keypoints
        if len(all_hand_peaks) > 0:
            if len(all_hand_peaks) > 0:  # Left hand
                for peak in all_hand_peaks[0]:
                    person_data["hand_left_keypoints_2d"].extend(
                        [float(peak[0]), float(peak[1]), 1.0 if peak[0] > 0 else 0.0]
                    )
            else:
                person_data["hand_left_keypoints_2d"].extend([0.0] * 63)
            if len(all_hand_peaks) > 1:  # Right hand
                for peak in all_hand_peaks[1]:
                    person_data["hand_right_keypoints_2d"].extend(
                        [float(peak[0]), float(peak[1]), 1.0 if peak[0] > 0 else 0.0]
                    )
            else:
                person_data["hand_right_keypoints_2d"].extend([0.0] * 63)

        json_data["people"].append(person_data)

    # Save JSON
    os.makedirs(output_path, exist_ok=True)
    with open(json_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)
    print(f"JSON saved at {json_path}")


if __name__ == "__main__":
    # Load models
    body_estimation, hand_estimation = load_model(use_hand=True)

    # 입력 및 출력 경로 설정
    input_path = 'input/image'
    output_path = 'pre-processing/openpose_json'

    # Ensure output directory exists
    os.makedirs(output_path, exist_ok=True)

    # 입력 디렉토리 내 모든 이미지 처리
    for image_name in os.listdir(input_path):
        if not image_name.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
            continue
        
        image_path = os.path.join(input_path, image_name)
        print(f'Processing: {image_path}')
        inference_and_save_json(image_path, body_estimation, hand_estimation, output_path)


JSON saved at pre-processing/openpose_json/00022_00_keypoints.json


# 2

In [16]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints
!mkdir inputs
!mkdir outputs

Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 722, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 722 (delta 74), reused 64 (delta 64), pack-reused 547 (from 1)
Receiving objects: 100% (722/722), 3.88 MiB | 21.14 MiB/s, done.
Resolving deltas: 100% (150/150), done.
/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing


In [17]:
!pip install gdown
!pip install ninja

In [18]:
import gdown
dataset = 'lip'

In [19]:
if dataset == 'lip':
    url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
elif dataset == 'atr':
    url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
elif dataset == 'pascal':
    url = 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'

output = 'checkpoints/final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=1de7ddae-287f-4df1-8ce8-ff59d0c4cf29
To: /Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/checkpoints/final.pth
100%|██████████| 267M/267M [00:08<00:00, 30.8MB/s] 


'checkpoints/final.pth'

In [ ]:
import os

# Input and output paths
input_dir = 'input/image'  # 디렉토리 내 모든 이미지 처리
output_path = 'pre-processing/image-parse'

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Process each image in the input directory
for image_name in os.listdir(input_dir):
    if not image_name.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
        continue

    input_path = os.path.join(input_dir, image_name)
    command = f"python3 simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/final.pth' --input-dir '{input_path}' --output-dir '{output_path}'"

    # Execute the command
    print(f"Running command: {command}")
    os.system(command)


Running command: python3 simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/final.pth' --input-dir '/Users/parkyunsu/Downloads/00022_00.jpg' --output-dir 'pre-processing/image-parse'


Traceback (most recent call last):
  File "simple_extractor.py", line 24, in <module>
    import networks
  File "/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/networks/__init__.py", line 3, in <module>
    from networks.AugmentCE2P import resnet101
  File "/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/networks/AugmentCE2P.py", line 21, in <module>
    from modules import InPlaceABNSync
  File "/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/modules/__init__.py", line 1, in <module>
    from .bn import ABN, InPlaceABN, InPlaceABNSync
  File "/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/modules/bn.py", line 10, in <module>
    from .functions import *
  File "/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/modules/functions.py", line 10, in <module>
    _backend = load(name="inplace_abn",
  File "/Users

256

# 3

In [27]:
!git clone https://github.com/facebookresearch/detectron2

Cloning into 'detectron2'...
remote: Enumerating objects: 15806, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15806 (delta 22), reused 38 (delta 9), pack-reused 15743 (from 1)
Receiving objects: 100% (15806/15806), 6.38 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (11518/11518), done.


In [26]:
!python -m pip install -e detectron2

ERROR: detectron2 is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [25]:
%cd detectron2/projects/DensePose
!pip install av>=8.0.3 opencv-python-headless>=4.5.3.56 scipy>=1.5.4

[Errno 2] No such file or directory: 'detectron2/projects/DensePose'
/Users/parkyunsu/.Trash/Self-Correction-Human-Parsing/Self-Correction-Human-Parsing/detectron2/projects/DensePose
zsh:1: 8.0.3 not found


In [24]:
import os

# Input and output paths
input_path = '/Users/parkyunsu/Downloads/00022_00.jpg'
output_path = 'pre-processing/image-densepose'

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Copy input image to a temporary directory for processing
temp_image_dir = "./image_path"
if os.path.exists(temp_image_dir):
    os.system(f"rm -rf {temp_image_dir}")
os.makedirs(temp_image_dir)

# Copy the input image to the temporary directory
os.system(f"cp {input_path} {temp_image_dir}/")

# Run DensePose inference
command = f"""
python apply_net.py show configs/densepose_rcnn_R_50_FPN_s1x.yaml \
https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl \
{temp_image_dir} dp_segm -v --output-dir {output_path}
"""

# Execute the command
print(f"Running command: {command}")
os.system(command)


Running command: 
python apply_net.py show configs/densepose_rcnn_R_50_FPN_s1x.yaml https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl ./image_path dp_segm -v --output-dir pre-processing/image-densepose



Traceback (most recent call last):
  File "apply_net.py", line 12, in <module>
    from detectron2.config import CfgNode, get_cfg
ModuleNotFoundError: No module named 'detectron2'


256

# 4

In [ ]:
!pip install carvekit_colab

In [ ]:
from carvekit.ml.files.models_loc import download_all
download_all();

In [ ]:
import os
import numpy as np
from PIL import Image
from carvekit.web.schemas.config import MLConfig
from carvekit.web.utils.init_utils import init_interface

# 설정
SHOW_FULLSIZE = False
PREPROCESSING_METHOD = "none"
SEGMENTATION_NETWORK = "u2net"
POSTPROCESSING_METHOD = "fba"
SEGMENTATION_MASK_SIZE = 320
TRIMAP_DILATION = 30
TRIMAP_EROSION = 5
DEVICE = 'cpu'

# 경로 설정
input_path = 'input/cloth'
output_path = 'pre-processing/cloth-mask'

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# CarveKit 설정
config = MLConfig(segmentation_network=SEGMENTATION_NETWORK,
                  preprocessing_method=PREPROCESSING_METHOD,
                  postprocessing_method=POSTPROCESSING_METHOD,
                  seg_mask_size=SEGMENTATION_MASK_SIZE,
                  trimap_dilation=TRIMAP_DILATION,
                  trimap_erosion=TRIMAP_EROSION,
                  device=DEVICE)

interface = init_interface(config)

# 입력 이미지 처리
imgs = [os.path.join(input_path, img) for img in os.listdir(input_path) if img.endswith(('.jpg', '.png'))]

# Segmentation 실행
images = interface(imgs)

# 결과 저장
for img_path, im in zip(imgs, images):
    img = np.array(im)
    idx = ((img[..., 0] == 0) & (img[..., 1] == 0) & (img[..., 2] == 0)) | \
          ((img[..., 0] == 130) & (img[..., 1] == 130) & (img[..., 2] == 130))

    img = np.ones(idx.shape) * 255
    img[idx] = 0
    im = Image.fromarray(np.uint8(img), 'L')
    im.save(os.path.join(output_path, os.path.basename(img_path).split(".")[0] + '.jpg'))


# 5

In [ ]:
import json
import os
from os import path as osp
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

# 입력으로 분할된 이미지(im_parse)와 포즈 데이터(pose_data)를 받아, 지정된 부위를 마스킹한 결과를 반환
def get_im_parse_agnostic(im_parse, pose_data, w=768, h=1024):
    label_array = np.array(im_parse)
    parse_upper = ((label_array == 5).astype(np.float32) +
                   (label_array == 6).astype(np.float32) +
                   (label_array == 7).astype(np.float32))  # 상체(5, 6, 7 레이블)와 목(10 레이블)을 마스킹할 영역으로 지정
    parse_neck = (label_array == 10).astype(np.float32)

    r = 10
    agnostic = im_parse.copy()  # 마스킹된 이미지

    # mask arms
    for parse_id, pose_ids in [(14, [2, 5, 6, 7]), (15, [5, 2, 3, 4])]:
        mask_arm = Image.new('L', (w, h), 'black')
        mask_arm_draw = ImageDraw.Draw(mask_arm)
        i_prev = pose_ids[0]
        for i in pose_ids[1:]:
            if (pose_data[i_prev, 0] == 0.0 and pose_data[i_prev, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
                continue
            mask_arm_draw.line([tuple(pose_data[j]) for j in [i_prev, i]], 'white', width=r * 10)
            pointx, pointy = pose_data[i]
            radius = r * 4 if i == pose_ids[-1] else r * 15
            mask_arm_draw.ellipse((pointx - radius, pointy - radius, pointx + radius, pointy + radius), 'white', 'white')
            i_prev = i
        parse_arm = (np.array(mask_arm) / 255) * (label_array == parse_id).astype(np.float32)
        agnostic.paste(0, None, Image.fromarray(np.uint8(parse_arm * 255), 'L'))

    # mask torso & neck
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_upper * 255), 'L'))
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_neck * 255), 'L'))

    return agnostic


if __name__ == "__main__":
    # 입력 및 출력 경로 설정
    input_image_path = 'input/image'  # 변경된 경로
    input_pose_path = 'pre-processing/openpose_json'
    input_parse_path = 'pre-processing/image-parse'
    output_path = 'pre-processing/image-parse-agnostic'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(input_image_path)):
        if not im_name.endswith('.jpg'):
            continue

        # 포즈 데이터 로드
        pose_name = im_name.replace('.jpg', '_keypoints.json')
        try:
            with open(osp.join(input_pose_path, pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data).reshape((-1, 3))[:, :2]
        except (IndexError, KeyError, FileNotFoundError) as e:
            print(f"Error processing pose data for {pose_name}: {e}")
            continue

        # 분할 이미지 로드
        parse_name = im_name
        try:
            im_parse = Image.open(osp.join(input_parse_path, parse_name))
        except FileNotFoundError as e:
            print(f"Error processing parse image for {parse_name}: {e}")
            continue

        # 마스킹 처리
        agnostic = get_im_parse_agnostic(im_parse, pose_data)

        # 결과 저장
        output_file_path = osp.join(output_path, parse_name)
        agnostic.save(output_file_path)


# 6

In [ ]:
import json
import os
from os import path as osp
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

def get_img_agnostic(img, parse, pose_data):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                  (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                   (parse_array == 12).astype(np.float32) +
                   (parse_array == 16).astype(np.float32) +
                   (parse_array == 17).astype(np.float32) +
                   (parse_array == 18).astype(np.float32) +
                   (parse_array == 19).astype(np.float32))

    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
    r = int(length_a / 16) + 1

    # mask arms
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r * 10)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx - r * 5, pointy - r * 5, pointx + r * 5, pointy + r * 5), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r * 10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx - r * 5, pointy - r * 5, pointx + r * 5, pointy + r * 5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx - r * 3, pointy - r * 6, pointx + r * 3, pointy + r * 6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r * 6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r * 6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r * 12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle((pointx - r * 7, pointy - r * 7, pointx + r * 7, pointy + r * 7), 'gray', 'gray')
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))

    return agnostic


if __name__ == "__main__":
    # 경로 설정
    input_image_path = 'input/image'  # 변경된 경로
    input_pose_path = 'pre-processing/openpose_json'
    output_path = 'pre-processing/agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(input_image_path)):
        if not im_name.endswith('.jpg'):
            continue

        # 포즈 데이터 로드
        pose_name = im_name.replace('.jpg', '_keypoints.json')
        try:
            with open(osp.join(input_pose_path, pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data).reshape((-1, 3))[:, :2]
        except (IndexError, KeyError, FileNotFoundError) as e:
            print(f"Error processing pose data for {pose_name}: {e}")
            continue

        # 이미지와 레이블 데이터 로드
        img = Image.open(osp.join(input_image_path, im_name))
        parse_name = im_name.replace('.jpg', '.png')
        try:
            parse = Image.open(osp.join(input_image_path.replace("image", "image-parse"), parse_name))
        except FileNotFoundError as e:
            print(f"Error processing parse image for {parse_name}: {e}")
            continue

        # 마스킹 처리
        agnostic = get_img_agnostic(img, parse, pose_data)

        # 결과 저장
        agnostic.save(osp.join(output_path, im_name))
